## scikeras Keras regressor integration

The integration of KerasRegressor in pipeline is limited. To use full keras capabilities, you may need to modify the pipeline.

In [1]:
from pathlib import Path
import glob
import gzip
import os
import pandas as pd

for folder in ['regression']:#, 'classif']:
    rootdir = Path('rawdata/'+folder)
    file_list = [f for f in rootdir.glob('**/*') if f.is_dir()]
    files_pattern = ["*Xcal*","*Ycal*","*Xval*","*Yval*"]
    for dir in file_list:
        training_name = str(dir)
        projdir = Path(dir)
        for f in files_pattern:
            file = next(projdir.glob(f))
            new_file = str(file).replace("rawdata","data2")
            new_file += '.gz'
            if os.path.exists(new_file):
                print("skipping", file)
                continue
            df = pd.read_csv(file, sep=";", header=0, skip_blank_lines=False)
            new_folder = str(file.parent).replace("rawdata","data2")
            if not os.path.exists(new_folder):
                os.makedirs(new_folder)
            df.to_csv(new_file, sep=";", index=False, compression="gzip")
            print(file, "done")

skipping rawdata\regression\Wood_Sustainability_511_Davrieux_RMSE0.092\Xcal.csv
skipping rawdata\regression\Wood_Sustainability_511_Davrieux_RMSE0.092\Ycal.csv
skipping rawdata\regression\Wood_Sustainability_511_Davrieux_RMSE0.092\Xval.csv
skipping rawdata\regression\Wood_Sustainability_511_Davrieux_RMSE0.092\Yval.csv


In [32]:
from pathlib import Path
import glob
import gzip
import os
import pandas as pd
import json

## un fichier par dataset > run_idx + datasheet + link vers runs & link vers tensorboard
## un fichier global json
## un export csv ligne=run colonnes=[datasetx(RMSE,R²)]
## correct RMSE <-> MSE

global_json = {}

def fill_json(v, revert):
    if revert:
        tmp = v['RMSE']
        v['RMSE'] = v['MSE']
        v['MSE'] = tmp

    d = v['dataset']
    if d not in global_json:
        global_json[d] = []
    global_json[d].append(v)

def parse(json_file, revert=False):
    print(json_file)

    with open(json_file) as json_file:
        results = json.load(json_file)
        if isinstance(results, list):
            for ar in results:
                v = ar[1]
                tmp = v['RMSE']
                fill_json(v, revert)
        else:
            for k, v in results.items():
                # print("DICT>", v['model'], v['dataset'], v['RMSE'])
                fill_json(v, revert)

# r=root, d=directories, f = files
for r, d, f in os.walk("results"):
    for folder in d:
        file_list = [f for f in Path(os.path.join(r, folder)).glob('*.json') if not f.is_dir()]
        for file in file_list:
            parse(file, True)
    
root_file_list = [f for f in Path("results").glob('*.json') if not f.is_dir()]
for file in file_list:
    parse(file, False)

### SORT runs per datasets
# for k in global_json:
    global_json[k] = sorted(global_json[k], key=lambda d: d['RMSE']) 

with open("GLOBAL.json", 'w') as fp:
    json.dump(global_json, fp, indent=4)




[WindowsPath('results/ALPINE_Calpine_424_Murguzur_results.json'), WindowsPath('results/ALPINE_C_424_Murguzur_results.json'), WindowsPath('results/ALPINE_Nalpine_552_Murguzur_results.json'), WindowsPath('results/ALPINE_N_552_Murguzur_results.json'), WindowsPath('results/ALPINE_Palpine_291_Murguzur_results.json'), WindowsPath('results/ALPINE_P_291_Murguzur_results.json'), WindowsPath('results/Cassava_TBC_3393_Sanchez_results.json'), WindowsPath('results/Cassava_TBC_3432_Shen_results.json'), WindowsPath('results/Cassava_TBC_3556_Davrieux_results.json'), WindowsPath('results/Cassava_TTC_3393_Sanchez_results.json'), WindowsPath('results/Cassava_TTC_3830_Davrieux_results.json'), WindowsPath('results/Eucalyptus_Density_1654_Chaix_results.json'), WindowsPath('results/LUCAS_SOCcropland_8731_Nocita_results.json'), WindowsPath('results/Meat_FatE1_215_Borggaard_results.json'), WindowsPath('results/Meat_FatE2_215_Borggaard_results.json'), WindowsPath('results/Meat_Fat_215_Borggaard_results.json'), 